# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [3]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import random
%matplotlib inline


SyntaxError: invalid syntax (<ipython-input-3-e3608ae59729>, line 8)

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines, [255,0,0],1)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
img_path = "test_images/"
img_list = os.listdir(img_path)

#reading in an image
image = mpimg.imread(img_path + img_list[5])

# image size
ysize = image.shape[0]
xsize = image.shape[1]

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image





Conversione Grayscale

In [ ]:
gray = grayscale(image)#cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
plt.imshow(gray,cmap='gray')

Applicazione gaussian blur

In [ ]:
# Define a kernel size and apply Gaussian smoothing
def apply_blur(_image):
    kernel_size = 1
    return gaussian_blur(_image, kernel_size) #cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

blur_gray = apply_blur(gray)
plt.imshow(blur_gray,cmap='gray')

Applicazione filtro di Canny

In [ ]:
# Define our parameters for Canny and apply
def apply_canny(image):
    low_threshold = 65
    high_threshold = 70
    return canny(image, low_threshold, high_threshold)

edges = apply_canny(blur_gray)
plt.imshow(edges,cmap='gray')

Aggiunta filtro in base a colore

In [ ]:
# Define our color criteria
def apply_color_filter(_image, _edges):
    red_threshold = 140
    green_threshold = 140
    blue_threshold = 255
    prova =  gaussian_blur(_image, 5)
    rgb_threshold = np.array([red_threshold, green_threshold, blue_threshold])
    #mask = cv2.inRange(_image, rgb_threshold, np.array([255, 255, 255]))
    mask = cv2.inRange(prova, np.array([0, 0, 0]),rgb_threshold )
    mask_inv = cv2.bitwise_not(mask)
    #mask = gaussian_blur(mask, 11)
    return cv2.bitwise_and(_edges,_edges, mask=mask_inv)
    #return cv2.bitwise_and(_image,_image, mask=mask_inv)

color_thresholds_image = apply_color_filter(image, edges)
#color_thresholds_image = edges
plt.imshow(color_thresholds_image,cmap='gray')

In [ ]:
def select_subimage(image):
    def compute_interest_areas():
            ysize = image.shape[0]
            xsize = image.shape[1]
            top = math.floor(0.58*ysize)
            left = math.floor(0.4*xsize)
            right = math.floor(0.6*xsize)
            return top, left, right
        

    top, left, right = compute_interest_areas()
      
    image = image[top:ysize,0:xsize]
    
    bottom_left = np.int32([0, ysize-top])
    bottom_right = np.int32([xsize, ysize-top])
    top_left = np.int32([left,0])
    top_right = np.int32([right,0])

    
    angoli_polig = [np.array([bottom_left, bottom_right, top_right, top_left], np.int32)]
    return region_of_interest(image, angoli_polig),top

edges_subimage, top_crop = select_subimage(color_thresholds_image)
#edges_subimage = gaussian_blur(edges_subimage, 11)
plt.imshow(edges_subimage, cmap='gray')

Selezione porzione di immagine

In [ ]:
# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
def hough_transform(_image):
    xsize = _image.shape[1]
    ysize = _image.shape[0]
    y_half = math.floor(ysize/2)
    
    rho = 2
    theta = 2*np.pi/180
    threshold = 50    
    min_line_length = 1
    max_line_gap = 40
    
    lines = hough_lines(_image, rho, theta, threshold, min_line_length, max_line_gap)
    
    return lines

lines = hough_transform(edges_subimage)

# GRAFICA
line_img = np.zeros((edges_subimage.shape[0], edges_subimage.shape[1], 3), dtype=np.uint8)
draw_lines(line_img, lines,thickness=6)

# Create a "color" binary image to combine with line image
color_edges = np.dstack((edges_subimage, edges_subimage, edges_subimage)) 

hough_image = weighted_img(line_img, color_edges)
plt.imshow(hough_image)


In [ ]:
# Sort of RANSAC alghoritm to select lines

def ransac(xsize, ysize, lines):    
    #threshold_distance = [1 ,30] # min, max
    
    if lines is None:
        return np.zeros((ysize, xsize, 3), dtype=np.uint8)
    
    inlierRatio = 0 # 0.3
    threshold = 20
    red_overlay_Yratio = 0.8
    
    points_number = 0
    for line in lines:
        A = line[0][0:2]
        B = line[0][2:4]
        points_number += np.linalg.norm([A[0]-B[0],A[1]-B[1]])
        
    
    results = []
    for sample in lines:
        A = sample[0][0:2]
        B = sample[0][2:4]
        inlierNum = 0
        fine = False
        
        # Find lane parameters (versor parallel, perpendicular + translation)
        kLine = B - A # two points relative distance
        #plt.plot([A[0],B[0]],[A[1],B[1]])
        normaVettore = np.linalg.norm(kLine)
        kLineNorm = kLine/normaVettore
        
        if math.fabs(kLineNorm[1]) < 0.5 or normaVettore < 5:
            #print("Skippato (corto e\o poco pendente) ")
            continue
        
        for normVector_tmp, q_tmp, _ in results:
            threshold_B = threshold
            threshold_A = threshold
            dist_A = math.fabs(normVector_tmp[0]*A[0]+normVector_tmp[1]*A[1]-q_tmp)
            dist_B = math.fabs(normVector_tmp[0]*B[0]+normVector_tmp[1]*B[1]-q_tmp)

            if (dist_A < threshold_A) or (dist_B < threshold_B):
                fine = True
                break
        
        if fine:
            continue
                 
        normVector = [-kLineNorm[1],kLineNorm[0]] #Ax+By+C=0 A=-kLineNorm(2),B=kLineNorm(1)

        q = normVector[0] * A[0] + normVector[1] * A[1]  

        
        # Find consensus
        inlierNum = 0
        inlayer_list = []
        y_max = 0
        y_min = ysize
        for line in lines:
            
            for x1, y1, x2, y2 in line:
                #print(x1,y1,x2,y2)
                
                kLine_local = [x2-x1, y2-y1]
        
                concordi = kLine_local[0]*normVector[0] + kLine_local[1]*normVector[1]
                #print("concordi scalar: \t{0}".format(concordi))
                
                if math.fabs(concordi) < 0.01:                    
                    dist_center = (A[0]-x1)*normVector[0] + (A[1]-y1)*normVector[1]
                    if math.fabs(dist_center<threshold):
                        intersections = [0,1]
                else:
                    
                    intersections = [] #init intersection
                    for _dir in [-1,1]:
                        # two points relative distance
                        b = np.array(A-[x1, y1]+[_dir*threshold*normVector[0], _dir*threshold*normVector[1]])
                        #print("A: {0}; point: ({1},{2}); b: {3}".format(A, x1, y1, b))
                        A_mat = np.array([[kLineNorm[0], kLine_local[0]], [kLineNorm[1], kLine_local[1]]])
                          
                        if concordi*threshold*_dir > 0: # concordi
                            _lambda = np.linalg.solve(A_mat, b)
                            if _lambda[1] < 0:
                                break
                        else:
                            _lambda = np.linalg.solve(A_mat, b)
                            if _lambda[1] > 1:
                                break
                        intersections.append(_lambda[1])
                    if len(intersections) == 2:
                        if intersections[0]>intersections[1]: #swap
                            tmp = intersections[0]
                            intersections[0] = intersections[1]
                            intersections[1] = tmp
                        
                        intersections[0] = max([intersections[0],0])
                        intersections[1] = min([intersections[1],1])
                        
                        #print(intersections)
                        lung = ((intersections[1]-intersections[0])*np.linalg.norm(kLine_local))
                        x_s = x1 + intersections[0] * kLine_local[0]
                        y_s = y1 + intersections[0] * kLine_local[1]
                        x_e = x1 + intersections[1] * kLine_local[0]
                        y_e = y1 + intersections[1] * kLine_local[1]
                        #print("y_s,y_e : {0},{1}".format(y_s,y_e))
                        if y_s<y_e:
                            #print("y_s < y_e")
                            if y_s<y_min:
                                y_min = y_s
                            if y_e>y_max:
                                y_max = y_e
                        else:
                            #print("y_e < y_s")
                            if y_e<y_min:
                                y_min = y_e
                            if y_s>y_max:
                                y_max = y_s
                        
                        inlayer_list.append([x_s,y_s, lung/2])
                        inlayer_list.append([x_e,y_e, lung/2])
                        
                        inlierNum += lung
                        #print("intersections: {0}".format(intersections))
                    #else:
                    #    print("No intersections")
        
        #print(y_max,y_min,inlierRatio*ysize)
        if not((y_max-y_min) >= np.int32(inlierRatio*ysize)):
            continue                             
        #result = [normVector, q, inlierNum/points_number]
        #results.append(result)
        
        # refine
        x_medio = 0
        y_medio = 0
        peso_tot = 0
        for x,y,peso in inlayer_list:
            x_medio += x*peso
            y_medio += y*peso        
        
        x_medio /= inlierNum
        y_medio /= inlierNum
        
        denom = 0
        num = 0
        for x,y,peso in inlayer_list:
            err_x = x-x_medio
            err_y = y-y_medio
            denom += peso*err_y*err_y            
            num += peso*err_x*err_y
         
        norma_vett = math.sqrt(pow(denom,2)+pow(num,2))
        normVector[0] = denom/norma_vett
        normVector[1] = -num/norma_vett
    
            
        q = x_medio*normVector[0] + y_medio*normVector[1]
        
        
              
        # Update the number of inliers and fitting model if better model is found     
        if np.isnan(q) or np.isnan(normVector[0]) or np.isnan(normVector[1]) :
            continue
        
        result = [normVector, q, inlierNum/points_number]
        results.append(result)
      
    detected_line_img = np.zeros((ysize, xsize, 3), dtype=np.uint8)
    for normVector, q, ratio in results:
        kLineNorm = [normVector[1],-normVector[0]]
        deltaY = red_overlay_Yratio*ysize
        scala = -deltaY/kLineNorm[1]
        delta=[]
        delta.append(scala*kLineNorm[0])
        delta.append(scala*kLineNorm[1])
        #print("norm delta: \t{0}".format(np.linalg.norm(delta)))
        #m = -normVector[1]/normVector[0]
        #q = q/normVector[0]
        P1 = [(-normVector[1]*ysize+q)/normVector[0],ysize]
        P2 = []
        P2.append(P1[0]+delta[0])
        P2.append(P1[1]+delta[1])
        #print(P1)
        #print(P2)
        line = [math.ceil(P1[0]),math.ceil(P1[1]),math.ceil(P2[0]),math.ceil(P2[1])]
        draw_lines(detected_line_img, [[line]], [255*ratio,0,0],12)
        #print(delta)
        #print([math.floor(P1[0]),math.floor(P2[0])],[math.floor(P1[1]),math.floor(P2[1])])
        
        #plt.plot([math.floor(P1[0]),math.floor(P2[0])],[math.floor(P1[1]),math.floor(P2[1])])
    #plt.show()
    return detected_line_img
        
detected_line_img = ransac(xsize, ysize-top_crop, lines)
# img, initial_img, α=0.8, β=1., λ=0.

tmp_img = np.zeros((ysize, xsize, 3), dtype=np.uint8)
tmp_img[top_crop:ysize,0:xsize] = detected_line_img
image_show  = weighted_img(tmp_img, image, α=0.5, β=20., λ=50.)
#plt.imshow(img_show)
plt.imshow(image_show)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):    
    xsize = image.shape[1]
    ysize = image.shape[0]
    gray = grayscale(image)
    blur_gray = apply_blur(gray)
    edges = apply_canny(blur_gray)
    color_thresholds_image = apply_color_filter(image, edges)
    #return color_thresholds_image
    edges_subimage,top_crop = select_subimage(color_thresholds_image)  
    #return edges_subimage
    lines = hough_transform(edges_subimage)
    detected_line_img = ransac(xsize, ysize-top_crop, lines)
    tmp_img = np.zeros((ysize, xsize, 3), dtype=np.uint8)
    tmp_img[top_crop:ysize,0:xsize] = detected_line_img
    image_res = weighted_img(tmp_img, image, α=0.5, β=20., λ=50.)
    return image_res

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

The current pipeline is structured as follow:
1. Convert the RGB image in grayscale,
```
gray = grayscale(image)
```
2. Apply on the gray image a blur filter to reduce the noise,
```
blur_gray = apply_blur(gray)
```
3. Detect in the blur_gray image the edges with canny,
```
edges = apply_canny(blur_gray)
```
4. Remove the edges of the shapes with dark colour,
```
color_thresholds_image = apply_color_filter(image, edges)
```
5. Crop the image and keep the part with the road lines (bottom),
```
edges,top_crop = select_subimage(color_thresholds_image)  
```
6. Apply the hough transform to detect the lines in the edges image as segments (pairs of points)
```
lines = hough_transform(edges)
```
7. Apply a ransac inspired function to filter the line segments and find the straight lines that better match the road lines
```
detected_line_img = ransac(xsize, ysize-top_crop, lines)
```

The first 6 steps are like the ones in the lectures, the only thing I want to stress is that the hough trasform is optimized for the recognition of straight lines and therefore bended lines are harder to detect.

The last step takes all the lines from the hough trasform and trys to infer the position of the roadlines returning a list of candidates. A possible way to improve this function is to filter the candidates and keep the 2 lines that are convergent with the intersection point near the middle of the image.


Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))